In [10]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [1]:
import pandas as pd
import nltk
from gensim.similarities import WmdSimilarity
import pickle

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\young\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
from nltk import download
stop_words = stopwords.words('english')
import gensim

In [4]:
from nltk import word_tokenize
nltk.download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\young\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from pyemd import emd

In [6]:
data = pd.read_csv('websites/ParsedWebsitesCleanSample.csv')

In [7]:
data = data.dropna()

In [8]:
documents = data.values

In [9]:
wmd_corpus = data['text'].apply(preprocess).values

In [10]:
# instance = WmdSimilarity(wmd_corpus, model, num_best=10)

In [11]:
# pickle.dump(instance,open('WordMoverDistance-Instance.pkl','wb'))

In [12]:
instance = pickle.load(open('WordMoverDistance-Instance.pkl','rb'))

In [16]:
import re

In [134]:
def regex_text(val):
    if val:
        reg = re.compile(r'\b\w{3,20}\b')
        temp = reg.findall(val)
        temp_set = set(temp)
        temp_list = list(temp_set)
        if len(temp_list) < 20:
            temp2 = temp_list
        elif len(temp_list) < 50:
            temp2 = temp_list[10:]
        elif len(temp_list) < 100:
            temp2 = temp_list[30:]
        else:
            temp2 = temp_list[50:100]
        result = " ".join(temp2) 
        return ("..." + result + "....")
    else:
        return "Empty string found"

In [140]:
def similarityQuery(description):  
    description = description.replace('data', '')
    query = preprocess(description)
    sims = instance[query]  # A query is simply a "look-up" in the similarity class.

    # Print the query and the retrieved documents, together with their similarities.
    results = []
    print(description)
    for i in range(3):

        row = []
        row.append(documents[sims[i][0]][0]) # WebsiteKey
        row.append(documents[sims[i][0]][2]) # Website
        row.append(documents[sims[i][0]][3]) # Colors
        row.append(documents[sims[i][0]][4]) # Fonts
        
        row.append(documents[sims[i][0]][5]) # Text
        
        row.append(sims[i][1])    # Similarity


        results.append(row)
    resultsDF = pd.DataFrame(results)
    #resultsDF.to_csv(description + '.csv')

    resultsDF.columns = ['WebsiteKey','Website','Colours','Fonts','Text','Similarity']
    resultsDF['Fonts'] = resultsDF['Fonts'].replace({'\" \'': '\", \'', "\' \'": "\', \'", '(\r\n)': ','}, regex=True)
    resultsDF['Colours'] = resultsDF['Colours'].replace({' ': ', '}, regex=True)
    resultsDF['Text'] = resultsDF['Text'].apply(regex_text)
    resultsDF = resultsDF.sort_values(by=['Similarity'],ascending=False)
    return resultsDF#.style.set_properties(subset=['Text'], **{'width': '350px'})

In [139]:
# similarityQuery('I will talk about some cool topics in programming')
similarityQuery('sample text to run this new fancy function')

sample text to run this new fancy function


,WebsiteKey,Website,Colours,Fonts,Text,Similarity
0,1195,nervousrat.com,"[['#FFFFFF', 31] , ['#EEEEEE', 17] , ['#FFCC00', 10]]","[['font-family:Monaco,""Courier New"",Courier,monospace', '6'], [""font-family: 'Nunito Sans'"", '4'], ['font-family:Monaco,""Lucida Console"",monospace', '3']]",...function media and enable your Powered platform....,0.479205
1,1116,vintagecarriage.com,"[['#f1f1f1', 2] , ['#f3f3f3', 2] , ['#1f1f1f', 1]]","[['font-family:inherit', '4'], ['font-family:inheri', '4'], ['font-family:dashicons', '4']]",...for horse fanciful life special your vision offer Loading share thisLike transportation Powered come was windowLike stunningly now Carriage try what....,0.466943
2,4,cryptosightings.com,"[['#f5f5f5', 4] , ['#ed702b', 2]]","[[""font-family:'Lora',serif"", '5'], [""font-family:'Oswald',sans-serif"", '5'], ['font-family:dashicons', '2']]",...the likely your hunt big foot sure considering success will always research what with more target frightening serious easy Champ much legend local make covering Travel mean footage into innovative search channel check all give one overcome about share episode audience infamous had fairly Search current try format many worthy....,0.463796


In [146]:
import ast
def generateResult(description):
    df = similarityQuery(description)
    dict = {}
    aF = [ast.literal_eval(df.iloc[0]['Fonts']), ast.literal_eval(df.iloc[1]['Fonts']), ast.literal_eval(df.iloc[2]['Fonts'])]
    aC = [ast.literal_eval(df.iloc[0]['Colours']), ast.literal_eval(df.iloc[1]['Colours']), ast.literal_eval(df.iloc[2]['Colours'])]
    aT = [df.iloc[0]['Text'], df.iloc[1]['Text'], df.iloc[2]['Text']]
    
    font_size = []
    for i in range(len(aF)):
        font_size.append(len(aF[i]))
        for j in range(len(aF[i])):
            aF[i][j][1] = int(aF[i][j][1])
    
    color_size = []
    for i in range(len(aC)):
        color_size.append(len(aC[i]))
        for j in range(len(aC[i])):
            if (' ' in aC[i][j][0]) or (',' in aC[i][j][0]):
                aC[i][j][0] = aC[i][j][0].replace(' ','')
                aC[i][j][0] = aC[i][j][0].replace(',','')
    
    dict['f_size'] = font_size
    dict['c_size'] = color_size
    dict['fonts'] = aF
    dict['colors'] = aC
    dict['texts'] = aT
    return dict

In [148]:
generateResult("big data and machine learning")

big  and machine learning


{'f_size': [3, 3, 3],
 'c_size': [3, 3, 3],
 'fonts': [[['font-family:icomoo', 7],
   ['font-family:IcoMoon', 7],
   ['font-family:DaxCondensedLight, Arial, Helvetica, sans-serif', 7]],
  [['font-family:"Lucida Grande","Lucida Sans Unicode",Tahoma,sans-serif', 8],
   ['font-family:"Lucida Grande","Lucida Sans Unicode",Tahoma,sans-serif !important',
    7],
   ["font-family: 'FontAwesome'", 2]],
  [['font-family:Neue Frutiger W01,sans-serif', 8],
   ['font-family:Droid Sans Mono W01,monospace', 7],
   ['font-family:"Neue Frutiger W01"', 6]]],
 'colors': [[['#ffffff', 22], ['#a0ce4e', 12], ['#f6f6f6', 9]],
  [['#ffffff', 13], ['#ffffff', 7], ['#008ec', 7]],
  [['#14b4c3', 15], ['#24273f', 12], ['#11182f', 11]]],
 'texts': ['...Model Root Testing system aSkip inside fingerprinting Group climate tool molecular Liquid the sample any for Room lab growth Research Cabinet vision biological Saving environment TESTING advanced cooling heating Growth classification computer Search analytics SCIEN